# async

In [ ]:
//// test

open testing

In [ ]:
open rust
open rust_operators

### base_let'

In [ ]:
inl base_let' x =
    let' x

## rust

### future

In [ ]:
nominal future t =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::future::Future<Output = $0>\")>]\n#endif\ntype std_future_Future<'T> = class end"
        $'' : $'std_future_Future<`t>'
    )

### future_pin

In [ ]:
type future_pin t = rust.pin (rust.box (rust.dyn' (future t)))

### future_pin_send

In [ ]:
type future_pin_send t = rust.pin (rust.box (rust.send (rust.dyn' (future t))))

### block_on_tokio

In [ ]:
inl block_on_tokio forall t. (fn : future_pin t) : t =
    inl runtime : infer =
        !\($'$"tokio::runtime::Builder::new_multi_thread().enable_all().build().unwrap()"')
    !\\(fn, $'"!runtime.handle().block_on($0)"')

### block_on_futures_lite

In [ ]:
inl block_on_futures_lite forall t. (fn : future_pin t) : t =
    !\\(fn, $'"futures_lite::future::block_on($0)"')

### block_on_futures

In [ ]:
inl block_on_futures forall t. (fn : future_pin t) : t =
    !\\(fn, $'"futures::executor::block_on($0)"')

### block_on_async_std

In [ ]:
inl block_on_async_std forall t. (fn : future_pin t) : t =
    !\\(fn, $'"async_std::task::block_on($0)"')

### block_on_tokio_send

In [ ]:
inl block_on_tokio_send forall t. (fn : future_pin_send t) : t =
    !\($'"tokio::runtime::block_on(!fn)"')

### stream_ext_tokio

In [ ]:
nominal stream_ext_tokio =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"tokio_stream::StreamExt\")>]\n#endif\ntype tokio_stream_StreamExt = class end"
        $'' : $'tokio_stream_StreamExt'
    )

### join_handle_tokio

In [ ]:
nominal join_handle_tokio t =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"tokio::task::JoinHandle<$0>\")>]\n#endif\ntype tokio_task_JoinHandle<'T> = class end"
        $'' : $'tokio_task_JoinHandle<`t>'
    )

### stream_collect_tokio

In [ ]:
inl stream_collect_tokio forall t u.
    (stream : t)
    : future_pin (am'.vec u)
    =
    !\($'"Box::pin(tokio_stream::StreamExt::collect(!stream))"')

### stream_collect_futures

In [ ]:
inl stream_collect_futures forall t u.
    (stream : t)
    : future_pin (am'.vec u)
    =
    !\($'"Box::pin(futures::stream::StreamExt::collect(!stream))"')

### stream_next_tokio

In [ ]:
inl stream_next_tokio forall t u.
    (stream : t)
    : future_pin (optionm'.option' u)
    =
    !\($'"let mut !stream = !stream"')
    !\($'"Box::pin(tokio_stream::StreamExt::next(&mut !stream))"')

### stream_filter_map_tokio

In [ ]:
inl stream_filter_map_tokio forall t u v.
    (fn : u -> optionm'.option' v)
    (stream : t)
    : infer' v
    =
    inl fn = join fn
    !\($'"tokio_stream::StreamExt::filter_map(!stream, |x| !fn(x))"')

### stream_filter_map_futures

In [ ]:
inl stream_filter_map_futures forall t u v.
    (fn : u -> optionm'.option' v)
    (stream : t)
    : infer' v
    =
    inl fn = join fn
    !\($'"futures::stream::StreamExt::filter_map(!stream, |x| async { !fn(x) })"')

### spawn_tokio

In [ ]:
inl spawn_tokio forall t. (fn : future_pin_send t) : join_handle_tokio t =
    !\($'"tokio::runtime::spawn(!fn)"')

### try_join_all

In [ ]:
nominal try_join_all t =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"futures::future::TryJoinAll<$0>\")>]\n#endif\ntype futures_future_TryJoinAll<'T> = class end"
        $'' : $'futures_future_TryJoinAll<`t>'
    )

inl try_join_all forall t. (x : am'.vec (future_pin (resultm.result' t sm'.std_string))) : try_join_all (future_pin (resultm.result' t sm'.std_string)) =
    inl x = join x
    !\($'"futures::future::try_join_all(!x)"')

### fuse_tokio

In [ ]:
nominal fuse_tokio t =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"tokio::prelude::stream::Fuse<$0>\")>]\n#endif\ntype tokio_prelude_stream_Fuse<'T> = class end"
        $'' : $'tokio_prelude_stream_Fuse<`t>'
    )

### fuse'

In [ ]:
type fuse' t = fuse_tokio t

### future_fuse

In [ ]:
inl future_fuse forall t. (x : future_pin t) : fuse' (future_pin t) =
    !\($'"futures::future::FutureExt::fuse(!x)"')

### join_all

In [ ]:
nominal join_all t =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"futures::future::JoinAll<$0>\")>]\n#endif\ntype futures_future_JoinAll<'T> = class end"
        $'' : $'futures_future_JoinAll<`t>'
    )

inl join_all forall t. (x : am'.vec (future_pin t)) : join_all (future_pin t) =
    inl x = join x
    !\($'"futures::future::join_all(!x)"')

### join_all_send

In [ ]:
inl join_all_send forall t. (x : am'.vec (future_pin_send t)) : join_all (future_pin_send t) =
    inl x = join x
    !\($'"futures::future::join_all(!x)"')

### join_handle'

In [ ]:
type join_handle' t = join_handle_tokio t

### await_handle

In [ ]:
inl await_handle forall t. (x : join_handle' t) : t =
    !\($'"!x.await"')

### await_all

In [ ]:
inl await_all forall t. (x : join_all (future_pin t)) : am'.vec t =
    !\($'"!x.await"')

### await_all_send

In [ ]:
inl await_all_send forall t. (x : join_all (future_pin_send t)) : am'.vec t =
    !\($'"!x.await"')

### try_await_all

In [ ]:
inl try_await_all forall t. (x : try_join_all (future_pin (resultm.result' t sm'.std_string))) : resultm.result' (am'.vec t) sm'.std_string =
    !\($'"!x.await"')

### try_await_all_send

In [ ]:
inl try_await_all_send forall t. (x : try_join_all (future_pin_send (resultm.result' t sm'.std_string))) : resultm.result' (am'.vec t) sm'.std_string =
    !\($'"!x.await"')

### await

In [ ]:
inl await forall t. (x : future_pin t) : t =
    !\($'"!x.await"')

### await

In [ ]:
inl await_send forall t. (x : future_pin_send t) : t =
    !\($'"!x.await"')

### into_iter

In [ ]:
nominal into_iter t =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"rayon::vec::IntoIter<$0>\")>]\n#endif\ntype rayon_vec_IntoIter<'T> = class end"
        $'' : $'rayon_vec_IntoIter<`t>'
    )

### into_par_iter

In [ ]:
inl into_par_iter forall t. (x : am'.vec t) : into_iter t =
    !\\(x, $'"rayon::iter::IntoParallelIterator::into_par_iter($0)"')

### par_iter

In [ ]:
inl par_iter forall t. (x : am'.vec t) : into_iter t =
    !\($'"rayon::iter::IntoParallelIterator::par_iter(!x)"')

### iter_map

In [ ]:
nominal iter_map t u =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"rayon::iter::Map<$0, _>\")>]\n#endif\ntype rayon_iter_Map<'T> = class end"
        $'' : $'rayon_iter_Map<`t>'
    )

### par_map

In [ ]:
inl par_map forall t u. (fn : t -> u) (ar : into_iter t) : iter_map (into_iter t) u =
    !\\((ar, fn), $'"rayon::iter::ParallelIterator::map($0, |x| $1(x))"')

### par_collect

In [ ]:
inl par_collect forall t u. (iter : iter_map (into_iter t) u) : am'.vec u =
    !\\(iter, $'"rayon::iter::ParallelIterator::collect($0)"')

### try_join_all_iter

In [ ]:
inl try_join_all_iter forall t. (x : am'.vec (future_pin_send (resultm.result' t sm'.std_string))) : try_join_all (future_pin_send (resultm.result' t sm'.std_string)) =
    inl x = join x
    !\($'"futures::future::try_join_all(!x)"')

### future_init

In [ ]:
inl future_init forall t. (move : bool) (x : () -> t) : infer' t =
    (!\($'"true; let __future_init = Box::pin(/*"') : bool) |> ignore
    if move
    then (!\($'"*/ async move { /*"') : bool) |> ignore
    else (!\($'"*/ async { /*"') : bool) |> ignore
    (!\($'"*/ //"') : bool) |> ignore

    inl x' = x ()
    // inl x' = join x'

    inl depth = 1, 0

    x' |> rust.fix_closure depth

    !\($'"__future_init"')

### new_future

In [ ]:
inl new_future forall t. (x : () -> t) : future_pin t =
    inl result = future_init false x
    !\($'"!result"')

### new_future_move

In [ ]:
inl new_future_move forall t. (x : () -> t) : future_pin t =
    inl result = future_init true x
    !\($'"!result"')

### new_future_send

In [ ]:
inl new_future_send forall t. (x : () -> t) : future_pin_send t =
    inl result = future_init false x
    !\($'"!result"')

### new_future_move_send

In [ ]:
inl new_future_move_send forall t. (x : () -> t) : future_pin_send t =
    inl result = future_init true x
    !\($'"!result"')

## fsharp

### async

In [ ]:
nominal async_python t =
    `(
        backend_switch `(()) `({}) {
            Python = (fun () => global "import asyncio") : () -> ()
        }
        $'' : $'any'
    )
type async_switch t =
    {
        Fsharp : $'Async<`t>'
        Python : async_python t
    }
nominal async t = $'backend_switch `(async_switch t)'

### task

In [ ]:
nominal task t =
    `(
        typecase t with
        | () => $'' : $'System.Threading.Tasks.Task'
        | _ => $'' : $'System.Threading.Tasks.Task<`t>'
    )

### new_async_unit

In [ ]:
inl new_async_unit forall t. (fn : () -> ()) : async t =
    join
        run_target_args' fn function
            | Fsharp _
            // | Rust _
            | TypeScript _
            | Python _ => fun fn =>
                    fun () =>
                        $'async {'
                        fun () =>
                            fn ()
                            real
                                typecase t with
                                | () => $'()' : ()
                                | _ => ()
                        |> indent
                        $'}' : ()
                    |> base_let'
            | Cuda _ => fun fn =>
                $'async def __new_async_unit__():'
                fun () =>
                    fn ()
                    $'""" new_async_unit'
                |> indent
                $'new_async_unit """'
                $'__new_async_unit__'
            | _ => fun _ => null ()

### new_async

In [ ]:
inl new_async forall t. (fn : () -> t) : async t =
    new_async_unit (fn >> ignore)

### new_task

In [ ]:
inl new_task forall t. (fn : () -> t) : task t =
    run_target_args' fn function
        | Fsharp _ => fun fn =>
            inl result : optionm'.option' (task t) = optionm'.none' ()
            $'let mutable _new_task_!result = !result '
            $'task {'
            fn () |> ignore
            $'}'
            $'|> fun x -> _new_task_!result <- Some x'
            $'match _new_task_!result with Some x -> x | None -> failwith "async.new_task / _new_task_!result=None"'
        | _ => fun _ => null ()

### await_task

In [ ]:
inl await_task forall t. (a : task t) : async t =
    run_target function
        | Fsharp _
        // | Rust _
        | TypeScript _
        | Python _ => fun () =>
            a |> $'Async.AwaitTask'
        | _ => fun () => null ()

### ignore

In [ ]:
inl ignore forall t. (a : async t) : async () =
    run_target function
        | Fsharp _
        // | Rust _
        | TypeScript _
        | Python _ => fun () =>
            a |> $'Async.Ignore'
        | _ => fun () => null ()

### run_synchronously

In [ ]:
inl run_synchronously forall t. (a : async t) : t =
    run_target function
        | Fsharp _
        // | Rust _
        | Python _ => fun () =>
            a |> $'Async.RunSynchronously'
        | Cuda (Native) => fun () =>
            $'asyncio.run(!a())'
        | _ => fun () => null ()

### start

In [ ]:
inl start (a : async ()) : () =
    run_target function
        | Fsharp _
        | Rust _
        | TypeScript _
        | Python _ => fun () =>
            a |> $'Async.Start'
        | _ => fun () => null ()

### start_child

In [ ]:
inl start_child forall t. (a : async t) : async (async t) =
    run_target function
        | Fsharp _
        | TypeScript _
        | Python _ => fun () =>
            a |> $'Async.StartChild'
        | _ => fun () => null ()

### start_child_timeout

In [ ]:
inl start_child_timeout forall t. (timeout : i32) (a : async t) : async (async t) =
    run_target function
        | Fsharp _
        | TypeScript _
        | Python _ => fun () =>
            $'Async.StartChild (!a, !timeout)'
        | _ => fun () => null ()

### start_immediate

In [ ]:
inl start_immediate forall t. (a : async t) : () =
    run_target function
        | Fsharp _
        // | Rust _
        | TypeScript _
        | Python _ => fun () =>
            a |> $'Async.StartImmediate'
        | _ => fun () => null ()

### start_with_continuations

In [ ]:
inl start_with_continuations forall t. (a : async t) : () =
    run_target_args' a function
        | Fsharp _
        | Rust _
        | TypeScript _
        | Python _ => fun a =>
            $'Async.StartWithContinuations (!a, ignore, ignore, ignore)'
        | _ => fun _ => null ()

### task_canceled_exception

In [ ]:
nominal task_canceled_exception = $'System.Threading.Tasks.TaskCanceledException'

### sleep

In [ ]:
inl sleep (ms : i32) : async () =
    run_target function
        | Fsharp _
        | Rust _
        | TypeScript _
        | Python _ => fun () =>
            ms |> $'Async.Sleep'
        | Cuda _ => fun () =>
            $'asyncio.sleep(!ms / 1000)'
        | _ => fun () => null ()

### do

In [ ]:
inl do (a : async ()) : () =
    backend_switch {
        Fsharp = fun () => $'do\! !a ' : ()
        Python = fun () => $'await !a ' : ()
    }

### let'

In [ ]:
inl let' forall t. (a : async t) : t =
    $'let\! !a = !a '
    $'!a '

### return_await

In [ ]:
inl return_await forall t. (a : async t) : () =
    backend_switch {
        Fsharp = fun () => $'return\! !a ' : ()
        Python = fun () => $'asyncio.run(!a())' : ()
    }

### return_await'

In [ ]:
inl return_await' forall t. (a : async t) : t =
    backend_switch {
        Fsharp = fun () => $'return\! !a ' : ()
        Python = fun () => $'await !a()' : ()
    }

### map

In [ ]:
inl map forall t u. (fn : t -> u) (a : async t) : async u =
    fun () =>
        inl x = a |> let'
        fn x |> return
    |> new_async_unit

### catch'

In [ ]:
inl catch' forall t e. (a : async t) : async (choice2' t e) =
    run_target function
        | Fsharp _
        // | Rust _
        | TypeScript _
        | Python _ => fun () =>
            a |> $'Async.Catch'
        | _ => fun () => null ()

### catch

In [ ]:
inl catch forall t e. (a : async t) : async (result t e) =
    a
    |> catch'
    |> map choice2_unbox
    |> map function
        | C1of2 result => Ok result
        | C2of2 ex => Error ex

### run_with_timeout_async

In [ ]:
let run_with_timeout_async forall t. (timeout : i32) (fn : async t) : async (option t) =
    run_target_args (fun () => timeout, fn) function
        | Fsharp _
        | Rust _
        | TypeScript _
        | Python _ => fun timeout, fn =>
            fun () =>
                fn
                |> start_child_timeout timeout
                |> let'
                |> catch
                |> map function
                    | Ok result => Some result
                    | Error ex when ex |> sm'.format_debug |> sm'.contains "System.TimeoutException" =>
                        trace Verbose
                            fun () => "async.run_with_timeout_async"
                            fun () => { timeout }
                        None
                    | Error (ex : exn) =>
                        trace Critical
                            fun () => "async.run_with_timeout_async**"
                            fun () => { timeout ex = ex |> sm'.format_exception }
                        None
                |> return_await
            |> new_async_unit
        | _ => fun _ => null ()

### run_with_timeout

In [ ]:
inl run_with_timeout timeout fn =
    fn
    |> run_with_timeout_async timeout
    |> run_synchronously

### cancellation_token

In [ ]:
inl cancellation_token () : async threading.cancellation_token =
    $'Async.CancellationToken'

In [ ]:
inl default_cancellation_token () : threading.cancellation_token =
    $'Async.DefaultCancellationToken'

### merge_cancellation_token_with_default_async

In [ ]:
inl merge_cancellation_token_with_default_async
    (token : threading.cancellation_token)
    : async threading.cancellation_token
    =
    fun () =>
        run_target function
            | Fsharp (Native) => fun () =>
                    inl ct = cancellation_token () |> let'
                    inl dct = default_cancellation_token ()
                    inl cts = threading.create_linked_token_source ;[ ct; dct; token ]
                    cts |> threading.cancellation_source_token |> return
            | _ => fun () => (null () : threading.cancellation_token) |> return
    |> new_async_unit

### with_trace_level

In [ ]:
inl with_trace_level forall t. level fn : _ t = new_async fun () =>
    inl trace_state = get_trace_state_or_init None
    inl old_trace_level = *trace_state.level
    inl trace_level = trace_state.level
    try_finally
        fun () =>
            trace_level <- level
            fn |> return_await
        fun () =>
            trace_level <- old_trace_level

### value_task

In [ ]:
nominal value_task = $'System.Threading.Tasks.ValueTask'

### value_task_as_task

In [ ]:
inl value_task_as_task (task : value_task) : task () =
    run_target function
        | Fsharp (Native) => fun () => $'!task.AsTask' ()
        | _ => fun () => null ()

### await_value_task_unit

In [ ]:
inl await_value_task_unit (task : value_task) : async () =
    task |> value_task_as_task |> await_task

## main

In [ ]:
inl main () =
    $'let merge_cancellation_token_with_default_async x = !merge_cancellation_token_with_default_async x' : ()